## Problem Statement
---

American Football is a very popular sport in the United States. Two of the biggest leagues that host the sport are the NFL, The National Football League, which is the professional organization that contain 32 teams across the USA, and NCAA College Football, which is the ameatur version of the NFL teams where over 100 Colleges and Universities have a team of student athelethes to compete in the sport. 

These two leagues have similar and contrasting reasons that makes the sport so amazing and competitive. Spectators are generally fans of one and not the other for varying reasons. I would like to make an attempt at bridging that gap. I plan to create a recommender system that recommends a comparative team to the average NFL fan based off of their favorite NFL team and favorite style of play (offense, defense, passing, etc.)

## Data Collection
---

The data I collected was scraped from the website sportsreference.com. This site is comprised of several sites that provides statistics for MANY sports. Their aim is to provide the most complete and accessible statistics that can be found anywhere.

This site works perfectly for me because it has all of the major stats for both college and NFL football dating back to the start of each of those leagues.

I collected offensive and defensive stats for college football dating back to 1985 and those same stats for the NFL dating back to 1970.

The process of gathering the data included using selenium and beautiful soup to scrape the website. The way that the website is structured is that it have several tables displayed that contains different variations of stats. sportsreference.com uses javascript to load these tables as you access the page. This required me to use selenium to access the tables of stats that I needed because simply using the requests library was returning unfavorable results. After gaining access to the tables I used beautiful soup to parse through the HTML to obtain the data. I utitlize this information to program several 'for loops' that loops through each row, table, and yearly page.

The scraper notebooks for both leagues yielded the following data:
* NFL: 1,477 observations/teams and 68 unique columns/statisitical categories
* College: 2,428 observations/teams and 66 unique columns/statistical categories

## Exploratory Data Analysis
---

The ultimate goal is to merge the 2 seperate DataFrame types into one with the same column length and column types. In order to achieve this there are several things that I must consider.
* The columns between both the NFL & College DataFrames have different orders. So I realigned them to be better suited to be joined later on.
* Also, the data or stats in each observation are not the same as well. (i.e. college provides there stats on an average per game basis and the nfl provides stats in total across the season, among other things. 
* Since we will be comparing the NFL to the College stats, I would like for my stats to be on a per game basis to make sure we are comparing apples to apples as much as possible. (NFL plays more games than College so their total stats would be more than college's).

I first divided each column of the NFL DataFrame by 16, which is number of games the NFL plays in per season, to achieve the per game average that the college DataFrame has. Next I next engineered two additional columns into the NFL DataFrame, using already available columns. (turnover & opponent completion percentage)

Finally I concatenated both sets of data and selected the 36 columns that matched between the NFL & College data and that I would like to explore in my recommender. I limited the years included in my DataFrame to 2014 to 2018 to represent the teams based of their most recent performance, due to the nature of turnover for each team in both leagues. 

As mentioned before, I am comparing two different leagues so their statisical peformance varies, on average. Since I am comparing these two, it is important to understand how and what categories differ. To tuly understand the distributions, I created a function that prints out 2 comparative box plots and 2 histograms of each statisical category of choice (a column in my DataFrame) between the NFL and college leagues.

The top findings from these distributions were: 
* Overall the spread of each statistic are generally wider for college than NFL, this make sense as there is only 32 NFL teams represented per year against over 100 college teams represented per year. This will naturally have more variation in their results.
* The average of most of the statistics are typically close between both leagues, with the exception of stats in the rushing category, offensive fumbles, & point totals.

<img src="./assets/rush%20att.png">

As seen in the image of above, there is a large descrepancy between the number of the rushing attempts that the NFL takes per game compared to the number of attempts that the college league takes. Because of the large difference in this category the other rushing categories follows this same trend.
* NFL has a mean of 26 rushing attempts.
* College has a mean of 39 rushing attempts.

<img src="./assets/fumbles.png">

* NFL has a mean of 1.27 fumbles
* College has a mean of 0.68 fumbles

This is an interesting find seeing that the NFL runs the ball less so you would assume that they have less opportunites to fumble, in addition to it being more teams in College translating to more teams having more opportunites for fumbles.

<img src="./assets/pass%20att.png">

* NFL has a mean of 35 pass attempts a game.
* College has a mean of 31 pass attempts a game

As far as passing attempts the mean of both league are closer together with the NFL attempting 4 more passes on average than college football. College football though have a wider spread with some teams attempting up to 60 passes a game, where as in the NFL the most attempts a game is 43.

## Recommender System
---

My recommender was created by calculating the pairwise distances between each team based on the specific statisical categories. The distances are returned as a value between 0 and 1. With 0 being most similar and 1 being most different. I evaluated the strength of my recommender based off of this. My goal is to get teams similarity as close to 0 as possible, in addition to capturing a realistic comparison based off of each team's performance in their perspective leagues in that given year.

I have seperate versions of my recommender based off of several factors.
* all NFL & College teams from 2014 & 2018. Data is not scaled.
* all NFL & College teams from 2014 & 2018. Data is scaled using Z-scores.
* scaled data and only 1 NFL team of interest compared to all college teams.
* data is scaled using Min-Max scaling and 1 NFL team of interest compared to all college teams.


**Version 1:**
The intial recommender that was created had some glaring drawbacks. Though, I was getting very promising low scores on the top teams being returned, I was only getting 100% NFL teams against NFL teams and college against college. This doesn't bode well, since my goal is to provide college teams that are similar to NFL teams.

**Version 2:**
to combat this phenomenon, I decided to scale my data using scikit learn's standard scaler to obtain zscores for each observation. By taking this action the similarity scores became worse and started to move a little further away from 0. 

**Version 3:**
With my similarity scores worsening and still obtaining no college team recommendations against NFL teams, I decided to compare 1 NFL team at a time to all of the college football teams and only being returned college similarities. The results of this was affective in being returned college teams as they were the only options in the DataFrame. As expected, the scores were not very good, ranging from 0.25 or more, depending on the team.

**Final version:**
Though I was able to have similarity scores for each team, whether they are college or NFL, it still doesn't account for the fact that the statisical performance between the top teams in the NFL and the top teams in college are vastly different. The same applies for the middle and low performers as well. This is evident in the rushing descrepancy I pointed out in earlier charts. 

I needed to find a way to get the teams on the same playing field statisically. I decided to go with Min-Max scaling, by using scikit learn's min-max scaler package. Min max scaling is different from z score scaling, because now the resulting values are in the range from 0 to 1. The formula is seen below: 

$score= \frac{xi - min(x)}{max(x) - min(x)}$

The steps I used to implement this scaling method is as follows:
1. Prior to merging the NFL & College DataFrames together I performed the min-max scaling to each individually.
    * This allowed me to both reduce the range of values in each DataFrame, in addition to having the values of one DataFrame affect the other. (example: the top percentile of college's rushing attempts not affecting the top percentile of NFL's rushing attempts.)
    * This resulted in have a range of values between 0 to 1 for each statisical category and also the top performers of a specific category in each league (i.e. rushing attempts) will be closer to 1 and the worse being closer to 0.
2. I finally combined the resulting scaled NFL & College DataFrames together to achieve consistent values across each column and each league. No statistic over powers another and no league over powers the other. The DataFrame can now be used to calculate the pairwise distances.

After obtaining a min-max scaled DataFrame, I now created a function that can use this matrix to achieve several tasks:
* Similar to version 3 of my recommender system, compare a single NFL team to the rest of the college football teams.
* being capable of comparing teams based off of specific statistical categories. Examples include:
    * Overall performance - includes all major categories
    * Offense - includes all categories related to offense (rushing, passing, touchdowns, etc.)
    * Defense - includes all categories related to defense (opponents rushing, passing, touchdowns, etc.)
    * Any other categories down to the specifics of only passing, only rushing.
* calculate the pairwise distances of the teams and return the 10 most similar teams.
* All of this will be determined by a few user inputs, into the function:
    * DataFrame, team of interest, year of interest, statistical category of interest.

## Flask App
---

I developed a web application that takes the foundations of my recommender system and inner function and translates this to the user. It takes 3 input from the user: NFL team of interest, season the team played in, and favorite style of play. After taking these 3 arguments, the user is provided with an instant recommended college team and the specific year of that team. 

## Conclusion
---

A fully functioning and well producing college team recommender system have been created. My goal is to increase the popularity of College Football in hopes, it one day reaches the level of the NFL. The popularity can be gauged in many ways. viewership, game attendance, fan sentiment, etc. I plan to evalute my recommender system's effectiveness by understanding if the teams I recommended gain more user activities by measuring click rate over time for links I provide and collecting user feedback. 

I am very pleased at the results of my college recommender system. I though do believe that it can be greatly improved upon in several ways: 
* I can add more features outside of statistics, like team success, star power, game attendence and ticket sales, etc. I can find more unique statistics to include as features that accurately depicts and compares a team statistical performance each year.
* I compared teams across 5 years (2014-2018), I can extend the range of teams to older periods, while making sure the change in team's performance is accounted for as the style of play changes over the decades.
* create recommenders that solely compare all NFL teams in history to one another and the same for college football. I already have the data readily available so this would be a instant modification to look into.

In addition to the recommender improvements, there are also some improvements that can be made to the application side. Including college team specific content to the user is a great start. When I recommend a team I can provide links to videos, games & events, links to team pages, and also more statistics to chew on. 
Mostly I would like to be able to provide information on what makes these teams similar and why my recommender thought so. Potentially doing some unsupervised learning can be a great tool to help with this. 